# 1. ThreadPoolExecutor

Thread pool is generally used in I/O-bounded tasks.


# Example: Wikipedia scraper
Scenario: for a given list of terms (`TERMS`) get the content of the first paragraph from Wikipedia article of each term.

Given:
 - `WIKI_URL`: base URL of English Wikipedia. In each task the term is appended at the end of this string (e.g. `'https://en.wikipedia.org/wiki/' + 'kite'` gives an [URL for an article about a kite](https://en.wikipedia.org/wiki/kite)).
 - `TERMS`: a list of terms to search.
 - `get_from_wiki`: a function which requests URL for a given term, checks the response status code and returns a tuple of status code and text of the first paragraph of given Wikipedia's article.
 - `get_first_paragraph`: a helper function for parsing the HTML content and extracting the first paragraph's text.
 - `timeit`: a decorator function for measuring the execution time of wrapped function.

In [1]:
from lxml import html
import time
from typing import *

import requests

WIKI_URL = 'https://en.wikipedia.org/wiki/'
TERMS = [
    'family',
    'measurement',
    'leader',
    'atmosphere',
    'possibility',
    'housing',
    'payment',
    'sympathy',
    'meal',
    'description',
    'intention',
    'community',
    'preference',
    'menu',
    'volume',
    'brewery',
    'abcdefgh',  # no article
    'assumption',
    'patience',
    'recipe',
]


def timeit(func):
    """Wraps the function for measuring its execution time."""
    
    def wrapped(*args, **kwargs):
        t_start = time.time()
        result = func(*args, **kwargs)
        print(f'Executed `{func.__name__}` in {(time.time() - t_start):.2f}s')
        return result
    
    return wrapped


def get_first_paragraph(html_text: str) -> str:
    """
    Returns a text from first paragraph of given html content.
    """
    tree = html.fromstring(html_text)
    paragraph = tree.find('body//p')
    if isinstance(paragraph, html.HtmlElement):
        return paragraph.text_content().strip()
    return ''
    

def get_from_wiki(term: str) -> Tuple[int, str]:
    """
    Returns the status code and text of first paragraph
    from wikipedia article in form of a tuple.
    """
    res = requests.get(WIKI_URL + term)
    status = res.status_code
    if status != 200:
        return status, ''
    return status, get_first_paragraph(res.content)


### Sample output for the [article about data scraping](https://en.wikipedia.org/wiki/Data_scraping) with use of the  `get_from_wiki` function:


In [2]:
code, text = get_from_wiki('data_scraping')
print(f'response code: {code}')
print(f'text: {text}')

response code: 200
text: Data scraping is a technique in which a computer program extracts data from human-readable output coming from another program.




## A standard sequential task which executes function for each term:


In [4]:
@timeit
def task_sequential(terms):
    return [get_from_wiki(term) for term in terms]



#### Test `task_sequential`:


In [5]:
result_sequential = task_sequential(TERMS)

for term, (code, text) in zip(TERMS, result_sequential):
    print(f'{term}, response code: {code}')
    print(text, '\n')

Executed `task_sequential` in 3.90s
family, response code: 200
In human society, family (from Latin: familia) is a group of people related either by consanguinity (by recognized birth) or affinity (by marriage or other relationship). The purpose of families is to maintain the well-being of its members and of society. Ideally, families would offer predictability, structure, and safety as members mature and participate in the community.[1] In most societies, it is within families that children acquire socialization for life outside the family, and acts as the primary source of attachment, nurturing, and socialization for humans.[2][3] Additionally, as the basic unit for meeting the basic needs of its members, it provides a sense of boundaries for performing tasks in a safe environment, ideally builds a person into a functional adult, transmits culture, and ensures continuity of humankind with precedents of knowledge. 

measurement, response code: 200
Measurement is the numerical quantita

## Parallel task: take 1
Parallelized task using the `ThreadPoolExecutor` and `submit()` methods. Returns a list of results.

In [6]:
from concurrent.futures import ThreadPoolExecutor


@timeit
def task_parallel(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        futures = [pool.submit(get_from_wiki, term) for term in terms]
    return [future.result() for future in futures]

#### Test `task_parallel`:


In [7]:
result_parallel = task_parallel(TERMS)
result_sequential == result_parallel  # same result?


Executed `task_parallel` in 1.03s


True

## Parallel task: take 2
Parallelized task using the `ThreadPoolExecutor` and `map()` method. Returns a list of results.

In [8]:
@timeit
def task_parallel_2(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        result = pool.map(get_from_wiki, terms)
    return list(result)

In [9]:
longer_list = TERMS * 3
_ = task_parallel_2(longer_list)
_ = task_parallel_2(longer_list, n_workers=30)

Executed `task_parallel_2` in 1.40s
Executed `task_parallel_2` in 1.36s


## Parallel task: take 3
Parallelized task using the `ThreadPoolExecutor` and `map()` method. Lazy approach.

In [10]:
@timeit
def task_parallel_3(terms, n_workers=10):
    with ThreadPoolExecutor(n_workers) as pool:
        yield from pool.map(get_from_wiki, terms)

In [11]:
res = task_parallel_3(TERMS, n_workers=2)
res
time.sleep(1)
for code, text in res:
    print(code)

Executed `task_parallel_3` in 0.00s
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
404
200
200
200
